# [MATH2504 Programming of Simulation, Analysis, and Learning Systems at The University of Queensland](https://courses.smp.uq.edu.au/MATH2504/)

## Semester 2, 2022

# Practical G: Towards project 2

In this practical we work towards project 2 by considering design patterns used in a discrete event simulation engines. See also [Unit 6](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/project2.html).


**The code in the cell below is from Unit 6.** It can serve as a basis for complex discrete event simulations.

In [1]:
using DataStructures
import Base: isless

abstract type Event end
abstract type State end

#Captures an event and the time it takes place
struct TimedEvent
    event::Event
    time::Float64
end

#Comparison of two timed events - this will allow us to use them in a heap/priority-queue
isless(te1::TimedEvent, te2::TimedEvent) = te1.time < te2.time

#This is an abstract function 
"""
It will generally be called as 
       new_timed_events = process_event(time, state, event)
It will generate 0 or more new timed events based on the current event
"""
function process_event end

#Generic events that we can always use
struct EndSimEvent <: Event end
struct LogStateEvent <: Event end

function process_event(time::Float64, state::State, es_event::EndSimEvent)
    println("Ending simulation at time $time.")
    return []
end

function process_event(time::Float64, state::State, ls_event::LogStateEvent)
    println("Logging state at time $time.")
    println(state)
    return []
end;

"""
The main simulation function gets an initial state and an initial event that gets things going.
Optional arguments are the maximal time for the simulation, times for logging events, and a call back function.
"""
function do_sim(init_state::State, init_timed_event::TimedEvent
                    ; 
                    max_time::Float64 = 10.0, 
                    log_times::Vector{Float64} = Float64[],
                    call_back = (time, state) -> nothing)

    #The event queue
    priority_queue = BinaryMinHeap{TimedEvent}()

    #Put the standard events in the queue
    push!(priority_queue, init_timed_event)
    push!(priority_queue, TimedEvent(EndSimEvent(), max_time))
    for lt in log_times
        push!(priority_queue,TimedEvent(LogStateEvent(), lt))
    end

    #initilize the state
    state = deepcopy(init_state)
    time = 0.0

    call_back(time, state)

    #The main discrete event simulation loop - SIMPLE!
    while true
        #Get the next event
        timed_event = pop!(priority_queue)

        #advance the time
        time = timed_event.time

        #Act on the event
        new_timed_events = process_event(time, state, timed_event.event) 

        #if the event was an end of simulation then stop
        isa(timed_event.event, EndSimEvent) && break 

        #The event may spawn 0 or more events which we put in the priority queue 
        for nte in new_timed_events
            push!(priority_queue, nte)
        end

        call_back(time, state)
    end
end;

In the Unit 6 lecture you see how to implement a simple M/D/1 queue based on the framework above. We'll now use this framework for a more complicated queueing system which is slightly closer to Project 2. We'll consider a **tandem queueing system** which is a sequence of queues where jobs arrive externally to the first queue and after service go to the second queue, and so fourth until exiting from the final queue and leaving the system.

The systems is already coded, but as part of the practical we'll use the code to experiment. The code can help with Project 2 as well.

We also introduce an additional type, `TandemNetworkParameters` which is used to store the general parameters of the system. Nodes in the system are numbered 1,...,`num_nodes`. Each node has a service rate (inverse of the mean) from `μ_array` and has a squared coefficient of variation (SCV) from `scv_array`. We use Gamma random variables.

### SCV, Exponential, and Gamma random variables

The SCV of a random variable is its variance divided by the mean squared. It is a normalized measure of the variability of the random variable. Exponential random variables have an SCV of $1$:

In [2]:
using Distributions

dist = Exponential(22.3)
scv(d)= var(d)/mean(d)^2
scv(dist)

1.0

In [3]:
#Exponential is a special case of Gamma with shape = 1
dist = Gamma(1,22.3)
scv(dist)

1.0

In [4]:
? Gamma #Note that sometimes the gamma distribution is parameterized not by scale but by rate (inverse of scale)

search: Gamma InverseGamma



```
Gamma(α,θ)
```

The *Gamma distribution* with shape parameter `α` and scale `θ` has probability density function

$$
f(x; \alpha, \theta) = \frac{x^{\alpha-1} e^{-x/\theta}}{\Gamma(\alpha) \theta^\alpha},
\quad x > 0
$$

```julia
Gamma()          # Gamma distribution with unit shape and unit scale, i.e. Gamma(1, 1)
Gamma(α)         # Gamma distribution with shape α and unit scale, i.e. Gamma(α, 1)
Gamma(α, θ)      # Gamma distribution with shape α and scale θ

params(d)        # Get the parameters, i.e. (α, θ)
shape(d)         # Get the shape parameter, i.e. α
scale(d)         # Get the scale parameter, i.e. θ
```

External links

  * [Gamma distribution on Wikipedia](http://en.wikipedia.org/wiki/Gamma_distribution)


In [5]:
dist = Gamma(2,3) #Shape of 2 and scale of 3

Gamma{Float64}(α=2.0, θ=3.0)

In [6]:
mean(dist) #The mean is shape*scale 

6.0

In [7]:
var(dist) #The variance is shape*scale^2

18.0

Hence for a gamma distribution  SCV = 1/shape.

In [8]:
dist = Gamma(0.1,3)
round(scv(dist), digits = 5)

10.0

In [9]:
#This is convenient in the queueing context

"""
A convenience function to make a Gamma distribution with desired rate (inverse of shape) and SCV.
"""
rate_scv_gamma(desired_rate::Float64, desired_scv::Float64) = Gamma(1/desired_scv, desired_scv/desired_rate)

dist = rate_scv_gamma(3., 0.5)
@show mean(dist)
@show scv(dist);

mean(dist) = 0.3333333333333333
scv(dist) = 0.5


In [10]:
dist

Gamma{Float64}(α=2.0, θ=0.16666666666666666)

# We now construct the tandem queueing network

In [11]:
struct TandemNetworkParameters
    num_nodes::Int              # The number of nodes (queues/servers) in the system
    λ::Float64                  # The external arrival rate to the first queue
    μ_array::Vector{Float64}    # The list of the rates of service in each of the queues.
    scv_array::Vector{Float64}  # A list of the squared coefficients of service times.
end

In [12]:
mutable struct TandemQueueNetworkState <: State
    queues::Vector{Int}             #A vector which indicates the number of customers in each queue
    params::TandemNetworkParameters #The parameters of the tandem queueing system
#     cumulative_arrivals::Vector{Int} #A count of the total number of arrivals to each queue
end
 
#External arrival to the firt queue
struct ExternalArrivalEvent <: Event end
 
struct EndOfServiceAtQueueEvent <: Event
    q::Int #The index of the queue where service finished
end

In [13]:
total_in_system(state::TandemQueueNetworkState) = sum(state.queues)

total_in_system (generic function with 1 method)

These two functions help generate random variables.

In [14]:
next_arrival_duration(s::State) = rand(Exponential(1/s.params.λ))
next_service_duration(s::State, q::Int) = rand(rate_scv_gamma(s.params.μ_array[q], s.params.scv_array[q]));

This is the main logic of the simulation.

**Note:** There is a **BUG** in one of these two functions below and this will cause the theoretical and simulation results to **not** agree. See end of prac.

In [23]:
function process_event(time::Float64, state::State, arrival_event::ExternalArrivalEvent)
    state.queues[1] += 1     #increase number in first queue
    new_timed_events = TimedEvent[]
 
    #prepare next arrival
    push!(new_timed_events, TimedEvent(ExternalArrivalEvent(), time + next_arrival_duration(state)))
 
    #if this is the only job on the server engage service
    state.queues[1] == 1 && push!(new_timed_events,
                                TimedEvent(EndOfServiceAtQueueEvent(1), time + next_service_duration(state,1)))
    return new_timed_events
end
 
#Process an end of service event
function process_event(time::Float64, state::State, eos_event::EndOfServiceAtQueueEvent)
    q = eos_event.q
    new_timed_events = TimedEvent[]
    
    state.queues[q] -= 1
    @assert state.queues[q] ≥ 0
    
    #if another customer in the queue then start a new service
    if state.queues[q] ≥ 1
        st = next_service_duration(state, q)
        push!(new_timed_events, TimedEvent(EndOfServiceAtQueueEvent(q), time + st)) 
    end
    
    #If there is a downstream queue
    if q < state.params.num_nodes
        state.queues[q+1] += 1 #move the job to the downstream queue
        
        #if the queue downstream was empty
        if state.queues[q+1] == 1 
            st = next_service_duration(state, q+1)
            push!(new_timed_events, TimedEvent(EndOfServiceAtQueueEvent(q+1), time + st)) 
        end
    end
    
    return new_timed_events
end

process_event (generic function with 4 methods)

# Experimenting with the simulation

We can now use the simulation code above to experiment.

## Long time horizon simulation

In [53]:
#This function runs the simulation and records a full trajectory
function do_experiment_long(;n=5, warm_up_time = 1e5, 
                            max_time = 1e6, scv = 1.0, 
                            λ = 1.0, μ_possibilities = [1.5, 2.0, 3.0])
    
    init_queues = fill(0,n)
    pars = TandemNetworkParameters(n, λ, rand(μ_possibilities,n), fill(scv,n))
    initial_state = TandemQueueNetworkState(init_queues, pars)
    
    #These four local variables (local for do_experiment_long)
    #Are avaialbe to record_stats() 
    
    #Stats recorded
    queues_integral = zeros(n)
    cumulative_arrivals = zeros(Int,n)
    
    #Helpers from previous call
    last_state = deepcopy(initial_state)  
    last_time = 0.0
    
    function record_stats(time::Float64, state::TandemQueueNetworkState) 
        if time ≥ warm_up_time  #Use a warmup time
            queues_integral += state.queues*(time-last_time) #recording integrals
            diff_in_queues = last_state.queues - state.queues
#             @show time, max.(diff_in_queues,0) #only keep cases where positive difference
            cumulative_arrivals += max.(diff_in_queues,0) #only keep cases where positive difference
        end
        
        last_time = time
        last_state = deepcopy(state) #copy notice that state is mutable 
        return nothing
    end

    do_sim(initial_state, 
        TimedEvent(ExternalArrivalEvent(),0.0), max_time = max_time, call_back = record_stats)
    queues_integral/(max_time-warm_up_time), cumulative_arrivals/(max_time-warm_up_time), pars
end

do_experiment_long (generic function with 1 method)

In [55]:
using Random
Random.seed!(1)
queue_means, arrival_rates, pars = do_experiment_long()#warm_up_time =0.0, max_time=5.0);

Ending simulation at time 1.0e6.


In [56]:
rates

5-element Vector{Float64}:
 1.0014133333333333
 1.0014144444444444
 1.0014155555555555
 1.0014166666666666
 1.0014144444444444

In [34]:
#The load on each individual queue is the external arrival rate divided by the service rate (as long as <1)
ρ_loads = round.(pars.λ ./ pars.μ_array,digits=8);

If all nodes have SCV=1.0 and for all nodes the load is less then 1, then there is a theoretical mean from queueing theory:

In [35]:
theor_mean(ρ) = round(ρ/(1-ρ),digits=3);

In [36]:
theor_means = theor_mean.(ρ_loads);

In [37]:
# import Pkg; Pkg.add("PrettyTables")

In [38]:
using PrettyTables
pretty_table(hcat(ρ_loads, theor_means, round.(queue_means,digits=3) ), header = ["ρ", "Theory", "Sim"] )

┌──────────┬────────┬───────┐
│        ρ │ Theory │   Sim │
├──────────┼────────┼───────┤
│ 0.666667 │    2.0 │ 2.055 │
│      0.5 │    1.0 │ 1.032 │
│ 0.333333 │    0.5 │ 0.527 │
│      0.5 │    1.0 │ 0.977 │
│ 0.333333 │    0.5 │ 0.528 │
└──────────┴────────┴───────┘


**Note:** You'll notice that the theoretical results and the simulation results **do not** agree. Even if you increase the simulation time that is still the case. There is a bug in the simulation code. Try to find it with inspection. It is a simple fix in one line. Once you fix it, if you run the simulation again, the theoretical and simulation results will agree.